In [1]:
import numpy as np
import pandas as p
import matplotlib.pyplot as plt
import cPickle
from sklearn.ensemble import RandomForestClassifier
documents_categories = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_categories.csv")
#clicks_test = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_test.csv", dtype={"display_id": int, "ad_id": int}))
#documents_meta = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_meta.csv")
#documents_entities = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_entities.csv")
promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/promoted_content.csv", dtype = {"ad_id":int, "document_id":int, "campaign_id":int, "advertiser_id":int})
#sample_submission = p.read_csv("/Users/astrachan/Desktop/outbrain/sample_submission.csv")
documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_topics.csv")
clicks_train = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("/Users/astrachan/Desktop/outbrain/events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : str, "geo_location" : str})
#page_views = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views.csv")
#page_views_sample = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views_sample.csv")

In [2]:
#add document to clicks_train
events.drop(events.columns[[1,3,4,5]],axis = 1,inplace=True)
clicks_train = clicks_train.merge(events, how='left', on='display_id')
del events

In [3]:
#split to validation and train, choosing display_id by random
display_ids = clicks_train.groupby(['display_id'])['display_id'].agg({'count' : 'count'}).reset_index().drop('count',axis = 1)
validation_displays = display_ids.sample(frac = 0.2)
clicks_train.reset_index(level=0, inplace=True)
validation = validation_displays.merge(clicks_train, how = 'inner', on = 'display_id')
train = clicks_train.drop(validation['index'])
del clicks_train

In [4]:
#split to validation and train, first 4/5 for train, 1/5 for validation
#leng = len(clicks_train.display_id.unique())
#start_display_id_validation = (4 * leng / 5)
#start_index_validation = clicks_train[clicks_train['display_id'] == start_display_id_validation].index.tolist()[0]
#validation = clicks_train[start_index_validation:]
#train = clicks_train[:start_index_validation]
#del clicks_train

In [5]:
#remove all confidence levels below 0.3, can decide on another criteria later on
#documents_entities = documents_entities[documents_entities.confidence_level > 0.3]
documents_categories = documents_categories[documents_categories.confidence_level > 0.3]
documents_topics = documents_topics[documents_topics.confidence_level > 0.3]

In [6]:
#drop confidence level
documents_categories.drop('confidence_level',axis=1, inplace=True)
documents_topics.drop('confidence_level',axis=1, inplace=True)

In [7]:
#prepare one-hot
topic_dummies = p.get_dummies(documents_topics['topic_id'],prefix="_ti")
categories_dummies = p.get_dummies(documents_categories['category_id'],prefix="_ci")

In [8]:
topics = documents_topics.drop('topic_id',1).join(topic_dummies)
categories = documents_categories.drop('category_id',1).join(categories_dummies)
categories = categories.groupby(['document_id']).sum().reset_index()
topics = topics.groupby(['document_id']).sum().reset_index()
del documents_categories, documents_topics, categories_dummies, topic_dummies

In [9]:
#create promoted, promoted content (ad information) with its' categories and topics
promoted_content.drop(promoted_content.columns[[2,3]],axis = 1,inplace=True)
promoted = promoted_content.merge(categories, how='left', on='document_id')
promoted = promoted.merge(topics, how='left', on='document_id').drop('document_id', axis=1)
del promoted_content
#all Nulls are of documents with no topic or category, equals to 0
promoted.fillna(0.0, inplace=True)

In [11]:
#get predictors, dummy chunk just for label names
chunk = train[:10].merge(promoted,how = 'left',on = 'ad_id')
chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
predictors=[x for x in chunk.columns if x not in ['display_id','ad_id','document_id','clicked']]

In [12]:
#number of iterations for training
print train.shape[0]/10**5

697


In [ ]:
#initiallize algorithm then fit it on chunks of size 10**5 - takes an hour or so
alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
i = 0
for chunk in np.array_split(train, train.shape[0]/10**5):
    alg.n_estimators += 3
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    alg.fit(chunk[predictors], chunk["clicked"])
    i += 1
    print i

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73


In [12]:
#save outputted tree with name [tree_name]
tree_name = 'secondTree'
with open('/Users/astrachan/Desktop/outbrain/' + tree_name, 'wb') as f:
    cPickle.dump(alg, f)

In [20]:
#number of iterations for validating
print (validation.shape[0]/10**5)

174


In [21]:
#run outputted tree on chunks of size 10**5 of validation
predY=[]
i = 0
for chunk in np.array_split(validation, validation.shape[0]/10**5):
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predY += chunk_pred
    i += 1
    print i

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174


In [ ]:
#commented and arranged up to here

In [34]:
predict = np.asarray(predY)
val_copy = validation.copy()
val_copy['predict'] = predict
#delete predict? validation? chunk? chunk_pred? predictors? predY? bentaim lo

In [46]:
#check how many of the predictions are true
#def true_and_true(row):
#    if (row['clicked'] == True) and (row['predict'] == 1.0):
#        return 1
#    else:
#        return 0
#val_copy['true_and_true'] = val_copy.apply(lambda row: true_and_true(row),axis=1)

In [53]:
#function evaluating how far were you from predicting right
val_copy.sort_values(['display_id', 'predict'], inplace=True, ascending=[True, False] )
val_copy["seq"] = np.arange(val_copy.shape[0])
Y_seq = val_copy[val_copy.clicked == 1].seq.values
Y_first = val_copy[['display_id', 'seq']].drop_duplicates(subset='display_id', keep='first').seq.values
Y_ranks = Y_seq - Y_first
score = np.mean(1.0 / (1.0 + Y_ranks))
print("MAP: %.12f" % score)

MAP: 1.000000000000


In [61]:
#assemble test and predict using built tree
test = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_test.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("/Users/astrachan/Desktop/outbrain/events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : str, "geo_location" : str})
events.drop(events.columns[[1,3,4,5]],axis = 1,inplace=True)
test = test.merge(events, how='left', on='display_id')
del events

In [65]:
#number of iterations for predicting
test.shape[0]/10**5

322

In [63]:
test.head()

,display_id,ad_id,document_id
0,16874594,66758,1792425
1,16874594,150083,1792425
2,16874594,162754,1792425
3,16874594,170392,1792425
4,16874594,172888,1792425


In [66]:
#predict
predYtest=[]
i = 0
for chunk in np.array_split(test, test.shape[0]/10**5):
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predYtest += chunk_pred
    i += 1
    print i

KeyError: "['clicked'] not in index"

In [ ]:
#build results into a submission